In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


# CHANGELOG
Compared to version 2:
- [x] Different way of encoding Ordinal columns solely using Pandas
- [x] Convert brand to Ordinal column using cost of backpack by brand over the average cost of backpack for all listed brands
- [ ] Use Optuna for hyperparameter tuning

In [35]:
train_orig = pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv', index_col=['id'])
test = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv', index_col=['id'])
train_xtra = pd.read_csv('/kaggle/input/playground-series-s5e2/training_extra.csv', index_col=['id'])

train = pd.concat([train_orig, train_xtra], axis=0)
train

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312
...,...,...,...,...,...,...,...,...,...,...
4194313,Nike,Canvas,NaN,3.0,Yes,Yes,Messenger,Blue,28.098120,104.74460
4194314,Puma,Leather,Small,10.0,Yes,Yes,Tote,Blue,17.379531,122.39043
4194315,Jansport,Canvas,Large,10.0,No,No,Backpack,Red,17.037708,148.18470


## Get numeric columns and categorical columns

In [36]:
num_df = train.select_dtypes(include=['number']).drop('Price', axis=1)
cat_df = train.select_dtypes(include=['object'])

num_cols = num_df.columns
cat_cols = cat_df.columns

num_df

,Compartments,Weight Capacity (kg)
id,,
0,7.0,11.611723
1,10.0,27.078537
2,2.0,16.643760
3,8.0,12.937220
4,1.0,17.749338
...,...,...
4194313,3.0,28.098120
4194314,10.0,17.379531
4194315,10.0,17.037708


## Get Ordinal columns and One Hot columns

In [37]:
ord_cols = ['Size', 'Laptop Compartment', 'Waterproof', 'Brand']
oh_cols = ['Material', 'Style', 'Color']

## Transform features

In [38]:
train['Brand'].unique()

array(['Jansport', 'Under Armour', 'Nike', 'Adidas', 'Puma', nan],
      dtype=object)

Going through amazon and finding backpacks of various brands, and the average cost of the first five on page. Obviously a better way would be to scrape the web and compile the data myself, but I am doing this for fun and not a job.

In [39]:
brand_map = {
    'Jansport' : (38 + 49 + 55 + 35 + 25) / 5,
    'Under Armour' : (47 + 46 + 35 + 25 + 45) / 5,
    'Nike' : (70 + 115 + 115 + 87 + 52) / 5,
    'Adidas' : (49 + 55 + 60 + 41 + 28) / 5,
    'Puma' : (28 + 59 + 18 + 38 + 34) / 5
}
average = sum(brand_map.values()) / 5

for k, v in brand_map.items():
    brand_map[k] = brand_map[k] / average

brand_map

{'Jansport': 0.8086469175340272,
 'Under Armour': 0.7926341072858287,
 'Nike': 1.7574059247397917,
 'Adidas': 0.9327461969575661,
 'Puma': 0.7085668534827861}

 *Define ordering for each value*

In [40]:
train['Size'] = train['Size'].map({'Small' : 0, 'Medium' : 1, 'Large' : 2})
train['Waterproof'] = train['Waterproof'].map({'No': 0, 'Yes' : 1})
train['Laptop Compartment'] = train['Laptop Compartment'].map({'No' : 0, 'Yes' : 1})
train['Brand'] = train['Brand'].map(brand_map)
train

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
0,0.808647,Leather,1.0,7.0,1.0,0.0,Tote,Black,11.611723,112.15875
1,0.808647,Canvas,0.0,10.0,1.0,1.0,Messenger,Green,27.078537,68.88056
2,0.792634,Leather,0.0,2.0,1.0,0.0,Messenger,Red,16.643760,39.17320
3,1.757406,Nylon,0.0,8.0,1.0,0.0,Messenger,Green,12.937220,80.60793
4,0.932746,Canvas,1.0,1.0,1.0,1.0,Messenger,Green,17.749338,86.02312
...,...,...,...,...,...,...,...,...,...,...
4194313,1.757406,Canvas,NaN,3.0,1.0,1.0,Messenger,Blue,28.098120,104.74460
4194314,0.708567,Leather,0.0,10.0,1.0,1.0,Tote,Blue,17.379531,122.39043
4194315,0.808647,Canvas,2.0,10.0,0.0,0.0,Backpack,Red,17.037708,148.18470


## Create transformers for each type of column

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

X = train.drop(columns = ['Price'])
y = train['Price']

cat_cols = train.columns.drop(['Weight Capacity (kg)', 'Price'])
oh_cols = ['Material', 'Style', 'Color']


# The following is for using XGBoost, which does not take strings
le = LabelEncoder()
for col in oh_cols:
    X[col] = le.fit_transform(X[col])
    test[col] = le.fit_transform(test[col])

categorical_transformer = Pipeline(
    steps = [
        ('impute', SimpleImputer(strategy = 'most_frequent'))
    ]
)

numerical_transformer = Pipeline(
    steps = [
        ('impute', SimpleImputer(strategy = 'mean')),
        ('scale', StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers = [
        ('cat', categorical_transformer, cat_cols),
        ('oh encode', OneHotEncoder(sparse_output = False), oh_cols),
        ('num', numerical_transformer, ['Weight Capacity (kg)'])
    ]
)

X = preprocessor.fit_transform(X)
X

array([[ 0.80864692,  1.        ,  1.        , ...,  0.        ,
         0.        , -0.91771889],
       [ 0.80864692,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.30057552],
       [ 0.79263411,  1.        ,  0.        , ...,  1.        ,
         0.        , -0.19600981],
       ...,
       [ 0.80864692,  0.        ,  2.        , ...,  1.        ,
         0.        , -0.13950869],
       [ 0.70856685,  0.        ,  1.        , ...,  0.        ,
         0.        ,  1.54508317],
       [ 0.79263411,  3.        ,  1.        , ...,  0.        ,
         0.        ,  0.72654453]])

In [42]:
test['Size'] = test['Size'].map({'Small' : 0, 'Medium' : 1, 'Large' : 2})
test['Waterproof'] = test['Waterproof'].map({'No': 0, 'Yes' : 1})
test['Laptop Compartment'] = test['Laptop Compartment'].map({'No' : 0, 'Yes' : 1})
test['Brand'] = test['Brand'].map(brand_map)
test = preprocessor.fit_transform(test)

test

array([[ 0.70856685,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.38419468],
       [ 1.75740592,  0.        ,  1.        , ...,  0.        ,
         0.        , -0.63536177],
       [ 0.9327462 ,  0.        ,  2.        , ...,  0.        ,
         0.        , -0.88702955],
       ...,
       [ 0.80864692,  2.        ,  0.        , ...,  0.        ,
         0.        ,  1.27635752],
       [ 0.70856685,  2.        ,  2.        , ...,  0.        ,
         0.        ,  1.11554046],
       [ 0.79263411,  1.        ,  1.        , ...,  0.        ,
         0.        ,  1.60413757]])

In [43]:
import lightgbm as lgb

lgb_data = lgb.Dataset(X, label=y)

params = {
    'objective': 'regression',  # Regression task
    'metric': 'rmse',          # Root Mean Squared Error
    'boosting_type': 'gbdt',   # Gradient Boosting Decision Tree
    'num_leaves': 31,          # Number of leaves in a tree
    'learning_rate': 0.05,     # Learning rate
    'feature_fraction': 0.9,   # Fraction of features used for training
    'bagging_fraction': 0.8,   # Fraction of data used for training
    'bagging_freq': 5,         # Frequency for bagging
    'verbose': -1              # Suppress output
}

# Computationally time consuming to run

cv_results = lgb.cv(
    params,                    # Hyperparameters
    lgb_data,                  # Dataset
    num_boost_round=200,       # Number of boosting rounds
    nfold=5,                   # Number of folds for cross-validation
    stratified=False,          # Stratified sampling (not needed for regression)
    shuffle=True,              # Shuffle data before splitting
    metrics='rmse',            # Evaluation metric
    seed=42,                    # Random seed for reproducibility
    callbacks=[
        lgb.early_stopping(stopping_rounds=5), # callbacks introduced in lgbm version 4
    ]
)

Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[199]	cv_agg's valid rmse: 38.8975 + 0.027467


In [44]:
best_num_boost_rounds = len(cv_results['valid rmse-mean'])
best_rmse = min(cv_results['valid rmse-mean'])
print(f"Best number of boosting rounds: {best_num_boost_rounds}")
print(f"Best RMSE: {best_rmse}")

Best number of boosting rounds: 199
Best RMSE: 38.89751114640386


In [45]:
final_model = lgb.train(
    params,
    lgb_data,
    num_boost_round=best_num_boost_rounds
)

# Save the model
final_model.save_model('lightgbm_final_model.txt')

In [46]:
loaded_model = lgb.Booster(model_file='lightgbm_final_model.txt')

In [47]:
y_pred = final_model.predict(test, num_iteration=final_model.best_iteration)
y_pred

array([80.61683004, 81.96845108, 82.15821753, ..., 83.07824868,
       82.33804606, 81.21347315])

In [50]:
test_idx = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv', index_col=['id']).index
test_idx

Index([300000, 300001, 300002, 300003, 300004, 300005, 300006, 300007, 300008,
       300009,
       ...
       499990, 499991, 499992, 499993, 499994, 499995, 499996, 499997, 499998,
       499999],
      dtype='int64', name='id', length=200000)

In [52]:
submission = pd.DataFrame({'Price': y_pred})
submission = submission.set_index(test_idx)
submission

,Price
id,
300000,80.616830
300001,81.968451
300002,82.158218
300003,80.905071
300004,78.127289
...,...
499995,80.030717
499996,77.619949
499997,83.078249


In [53]:
submission.to_csv('submission.csv')